In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [5]:
# Chemin du projet et du fichier
project_root = r'C:\Users\Marouane\ImmoPredict'
dvf_file = "valeursfoncieres-2024.txt"  # Ajustez si nécessaire
dvf_path = os.path.join(project_root, 'data', 'raw', dvf_file)

In [6]:
# Chargement des données
print("Chargement des données DVF...")
df = pd.read_csv(dvf_path, sep='|', encoding='utf-8', low_memory=False)
print(f"Dimensions initiales: {df.shape}")

Chargement des données DVF...
Dimensions initiales: (3458643, 43)


In [7]:
# 1. Suppression des colonnes avec trop de valeurs manquantes (>95%)
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({'Nombre': missing_values, 'Pourcentage (%)': missing_percent})

In [8]:
# Colonnes à conserver malgré un taux élevé de valeurs manquantes
critical_cols = ['Valeur fonciere', 'Surface reelle bati', 'Nombre pieces principales', 
                'Type local', 'Code departement', 'Code commune', 'Surface terrain']

In [9]:
# Identification des colonnes à supprimer
cols_to_drop = [col for col in df.columns 
                if missing_percent[col] > 95 and col not in critical_cols]

In [10]:
# Suppression des colonnes
df_cleaned = df.drop(columns=cols_to_drop)
print(f"Nombre de colonnes supprimées: {len(cols_to_drop)}")
print(f"Dimensions après suppression des colonnes: {df_cleaned.shape}")

Nombre de colonnes supprimées: 19
Dimensions après suppression des colonnes: (3458643, 24)


In [11]:
# 2. Filtrage des lignes avec les informations essentielles
# Nous allons conserver uniquement les lignes qui ont des valeurs pour:
# - Valeur fonciere (prix)
# - Surface reelle bati OU Surface terrain
# - Type local (si disponible)

# Création d'un masque pour les lignes à conserver
mask_price = df_cleaned['Valeur fonciere'].notna()
mask_surface = df_cleaned['Surface reelle bati'].notna() | df_cleaned['Surface terrain'].notna()

In [12]:
# Application du filtre
df_filtered = df_cleaned[mask_price & mask_surface].copy()
print(f"Dimensions après filtrage des lignes: {df_filtered.shape}")

Dimensions après filtrage des lignes: (3260673, 24)


In [13]:
# 3. Conversion des types de données
# Conversion des valeurs numériques
numeric_cols = ['Valeur fonciere', 'Surface reelle bati', 'Nombre pieces principales', 'Surface terrain']

for col in numeric_cols:
    if col in df_filtered.columns:
        # Remplacer les virgules par des points pour les nombres décimaux
        if df_filtered[col].dtype == 'object':
            df_filtered[col] = df_filtered[col].str.replace(',', '.').astype(float)
        # Conversion en float si ce n'est pas déjà le cas
        df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce')

In [14]:
# 4. Filtrage des valeurs aberrantes
# Fonction pour filtrer les valeurs aberrantes
def filter_outliers(df, column, min_val=None, max_percentile=0.99):
    if column not in df.columns or df[column].isna().all():
        return df
    
    filtered_df = df.copy()
    
    # Filtre inférieur si spécifié
    if min_val is not None:
        filtered_df = filtered_df[filtered_df[column] >= min_val]
    
    # Filtre supérieur basé sur le percentile
    max_val = filtered_df[column].quantile(max_percentile)
    filtered_df = filtered_df[filtered_df[column] <= max_val]
    
    return filtered_df

In [15]:
# Application des filtres
df_filtered = filter_outliers(df_filtered, 'Valeur fonciere', min_val=10000)
df_filtered = filter_outliers(df_filtered, 'Surface reelle bati', min_val=10)
df_filtered = filter_outliers(df_filtered, 'Surface terrain', min_val=0)

print(f"Dimensions après filtrage des valeurs aberrantes: {df_filtered.shape}")

Dimensions après filtrage des valeurs aberrantes: (636775, 24)


In [16]:
# 5. Création de nouvelles variables
# Prix au m²
if 'Surface reelle bati' in df_filtered.columns and not df_filtered['Surface reelle bati'].isna().all():
    mask = df_filtered['Surface reelle bati'] > 0
    df_filtered.loc[mask, 'Prix_m2_bati'] = df_filtered.loc[mask, 'Valeur fonciere'] / df_filtered.loc[mask, 'Surface reelle bati']
    
    # Filtrage des prix au m² aberrants
    df_filtered = filter_outliers(df_filtered, 'Prix_m2_bati', min_val=100)

In [17]:
# Prix au m² du terrain
if 'Surface terrain' in df_filtered.columns and not df_filtered['Surface terrain'].isna().all():
    mask = df_filtered['Surface terrain'] > 0
    df_filtered.loc[mask, 'Prix_m2_terrain'] = df_filtered.loc[mask, 'Valeur fonciere'] / df_filtered.loc[mask, 'Surface terrain']
    
    # Filtrage des prix au m² aberrants
    df_filtered = filter_outliers(df_filtered, 'Prix_m2_terrain', min_val=10)

In [18]:
# 6. Encodage des variables catégorielles
if 'Type local' in df_filtered.columns and not df_filtered['Type local'].isna().all():
    # Filtrer pour ne garder que les types de biens pertinents
    valid_types = ['Appartement', 'Maison', 'Dépendance']
    mask_valid_types = df_filtered['Type local'].isin(valid_types)
    df_filtered = df_filtered[mask_valid_types]
    
    # One-hot encoding
    df_filtered = pd.get_dummies(df_filtered, columns=['Type local'], prefix='Type')

print(f"Dimensions finales: {df_filtered.shape}")

Dimensions finales: (586349, 27)


In [19]:
# 7. Sauvegarde du dataset nettoyé
processed_path = os.path.join(project_root, 'data', 'processed', 'dvf_clean.csv')
df_filtered.to_csv(processed_path, index=False)
print(f"Données nettoyées sauvegardées dans: {processed_path}")

Données nettoyées sauvegardées dans: C:\Users\Marouane\ImmoPredict\data\processed\dvf_clean.csv


In [20]:
# 8. Analyse du dataset nettoyé
print("\nAperçu des données nettoyées:")
display(df_filtered.head())

print("\nStatistiques descriptives:")
display(df_filtered.describe())

print("\nValeurs manquantes restantes:")
missing_after = df_filtered.isnull().sum()
missing_percent_after = (missing_after / len(df_filtered)) * 100
missing_df_after = pd.DataFrame({'Nombre': missing_after, 'Pourcentage (%)': missing_percent_after})
display(missing_df_after[missing_df_after['Nombre'] > 0].sort_values('Pourcentage (%)', ascending=False))


Aperçu des données nettoyées:


,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,...,Nombre de lots,Code type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain,Prix_m2_bati,Prix_m2_terrain,Type_Appartement,Type_Maison
78,1,03/01/2024,Vente,94500.0,32.0,RUE,0325,DES PETITES FONTAINES,1150.0,LAGNIEU,...,0,2.0,74.0,3.0,S,65.0,1277.027027,1453.846154,True,False
80,1,03/01/2024,Vente,94500.0,32.0,RUE,0325,DES PETITES FONTAINES,1150.0,LAGNIEU,...,0,2.0,32.0,2.0,S,65.0,2953.125000,1453.846154,True,False
81,1,03/01/2024,Vente,220000.0,5043.0,NaN,B152,AU MOLLARD,1640.0,BOYEUX-SAINT-JEROME,...,0,1.0,40.0,1.0,S,488.0,5500.000000,450.819672,False,True
82,1,03/01/2024,Vente,220000.0,14.0,MTE,0086,DU FOUR,1640.0,BOYEUX-SAINT-JEROME,...,0,1.0,80.0,2.0,S,858.0,2750.000000,256.410256,False,True
86,1,03/01/2024,Vente,72000.0,16.0,RUE,0045,CENTRALE,1150.0,LAGNIEU,...,0,1.0,121.0,4.0,S,115.0,595.041322,626.086957,False,True



Statistiques descriptives:


,No disposition,Valeur fonciere,No voie,Code postal,Code commune,No plan,Nombre de lots,Code type local,Surface reelle bati,Nombre pieces principales,Surface terrain,Prix_m2_bati,Prix_m2_terrain
count,586349.000000,5.863490e+05,584618.000000,586304.000000,586349.000000,586349.000000,586349.000000,586349.000000,586349.000000,586349.000000,586349.000000,586349.000000,586349.000000
mean,1.084738,3.348446e+05,666.752859,49847.798964,220.267844,404.659786,0.002988,1.131773,96.149029,3.966702,761.417848,4405.024044,813.420910
std,1.090120,6.185163e+05,1821.305507,26381.513440,174.503569,547.794953,0.054581,0.338244,46.452765,1.600371,1024.911105,9869.236889,1073.401363
min,1.000000,1.000000e+04,1.000000,1000.000000,1.000000,1.000000,0.000000,1.000000,10.000000,0.000000,2.000000,100.000000,10.000000
25%,1.000000,1.300000e+05,8.000000,29160.000000,81.000000,92.000000,0.000000,1.000000,67.000000,3.000000,259.000000,1478.260870,225.000000
50%,1.000000,2.100000e+05,23.000000,49800.000000,183.000000,220.000000,0.000000,1.000000,90.000000,4.000000,500.000000,2272.800000,452.399685
75%,1.000000,3.340000e+05,101.000000,72700.000000,316.000000,493.000000,0.000000,1.000000,118.000000,5.000000,850.000000,3613.068182,931.818182
max,39.000000,1.287150e+07,9999.000000,97490.000000,909.000000,8395.000000,1.000000,2.000000,622.000000,198.000000,10800.000000,134408.602151,9444.100000



Valeurs manquantes restantes:


,Nombre,Pourcentage (%)
2eme lot,586349,100.000000
Surface Carrez du 1er lot,585192,99.802677
1er lot,584597,99.701202
Type de voie,82453,14.062103
No voie,1731,0.295217
Code postal,45,0.007675
Section,23,0.003923
Voie,4,0.000682
